# Números naturales. Inducción. Recursividad.

## Naturales

En `python` podemos utilizar `isinstance` para determinar si un objeto es un entero

In [1]:
isinstance(4,int)

True

In [2]:
isinstance([3,4],int)

False

Podemos con esta función definir otra que detecte si un objeto es un número es natural

In [3]:
def isnatural(n):
    if not isinstance(n,int):
        return false
    return n>=0

In [4]:
isnatural(3)

True

In [5]:
isnatural(-3)

False

La funciones sucesor y predecesor quedarían como sigue

In [6]:
sucesor = lambda x: x+1

In [7]:
sucesor(2)

3

In [8]:
def prec(n):
    if not isnatural(n):
        raise TypeError("El argumento debe ser un número natural")
    if n==0: 
        raise ValueError("El 0 no tiene predecesor")
    return n-1

In [9]:
prec(1)

0

Con esto podemos definir una suma

In [10]:
def suma(m,n):
    if n == 0:
        return m
    return sucesor(suma(m,prec(n)))

In [11]:
suma(2,3)

5

## Sucesiones, inducción

Con la librería `sympy` podemos hacer cálculo simbólico. En particular, podemos calcular el valor de algunas sumas con parámetros.

In [12]:
from sympy import *

Cuando vamos a utilizar un símbolo tenemos que declararlo primero. Para el ejemplo que sigue vamos a utilizar `n` como entero e `i` como contador 

In [13]:
n, i = symbols("n, i", integer = True)

### Algunas sumatorias

Calculemos por ejemplo $\sum_{i=1}^n i$

In [14]:
s = Sum(i,(i,1,n))

In [15]:
s

Sum(i, (i, 1, n))

Si queremos calcular el "valor" de esta sumatoria, podemos utilizar el método `doit`

In [16]:
s.doit()

n**2/2 + n/2

In [17]:
pprint(_)

 2    
n    n
── + ─
2    2


Y una suma de potencias

In [18]:
s=Sum(i**30,(i,1,n))

In [19]:
s.doit()

n**31/31 + n**30/2 + 5*n**29/2 - 203*n**27/6 + 1131*n**25/2 - 16965*n**23/2 + 216775*n**21/2 - 2304485*n**19/2 + 19959975*n**17/2 - 137514723*n**15/2 + 731482225*n**13/2 - 31795091601*n**11/22 + 8053785025*n**9/2 - 102818379585*n**7/14 + 15626519181*n**5/2 - 23749461029*n**3/6 + 8615841276005*n/14322

In [20]:
a = Symbol("a")

In [21]:
Sum(a**i,(i,1,n)).doit()

Piecewise((n, a == 1), ((a - a**(n + 1))/(-a + 1), True))

### Inducción

Veamos cómo podemos utilizar `sympy` para hacer algunos ejemplos de inducción

Por ejemplo, veamos que para todo $6\mid 7^n-1$ para todo $n\in \mathbb N$. Empezamos definiendo una función que nos devuelva $7^n-1$

In [22]:
f = lambda n: 7**n -1

Primero veamos qué valor tiene en el 0

In [23]:
f(0)

0

Si por hipótesis de inducción $f(n)$ es un múltiplo de 6, entonces para probar que $f(n+1)$ también lo es, bastará demostrar que la diferencia $f(n+1)-f(n)$ es un múltiplo de 6

In [24]:
simplify(f(n+1)-f(n))

6*7**n

In [25]:
f = lambda n: 7**(2*n)+16*n-1

In [26]:
f(0)

0

In [27]:
simplify((f(n+1)-f(n)) % 64)

16*Mod(3*7**(2*n) + 1, 4)

Por tanto, si $3\times 7^{2n}+1$ es un múltiplo de $4$, habremos acabado

In [28]:
g = lambda n: 3*7**(2*n)+1

In [29]:
g(0)

4

In [30]:
simplify((g(n+1)-g(n))%4)

0

### Recursividad e iteración

Veamos cómo las funciones recursivas se pueden acelerar usando memorización de los pasos anteriores

In [31]:
fib = lambda n: n if n<2 else fib(n-2)+fib(n-1)

In [32]:
fib(10)

55

In [33]:
fibonacci(10)

55

In [34]:
import time


In [35]:
start=time.time()
fib(30)
time.time()-start


0.42685985565185547

In [36]:
start=time.time()
fibonacci(30)
time.time()-start

0.0003609657287597656

Un posible solución a este problema es ir almacenando los resultados previos de cálculo, lo cual es conocido como [memoization](http://stackoverflow.com/questions/1988804/what-is-memoization-and-how-can-i-use-it-in-python)

In [37]:
import functools

In [38]:
@functools.lru_cache(maxsize=None)
def fibo(num):
    if num < 2:
        return num
    else:
        return fibo(num-1) + fibo(num-2)

In [39]:
start=time.time()
fibo(30)
time.time()-start

0.00010895729064941406

## Resolución de ecuaciones recursivas

Las ecuaciones en recurrencia se pueden resolver con el comando `rsolve` de `sympy`

Empecemos con un ejemplo:
- $a(0)=0$,
- $a(1)=1$,
- $a(n+2)=5a(n+1)-6a(n)$.

Empezamos declarando `a` como función

In [40]:
a = Function('a')

In [41]:
rsolve(a(n+2)-5*a(n+1)+6*a(n),a(n))

2**n*C0 + 3**n*C1

In [42]:
rsolve(a(n+2)-5*a(n+1)+6*a(n),a(n), {a(0):0,a(1):1})

-2**n + 3**n

Vamos a resolverlo mediante el polinomio característico

In [43]:
x=Symbol("x")
solve(x**2-5*x+6)

[2, 3]

Y por tanto la solución general será de la forma $a_n=u 2^n + v 3^n$, para ciertas constantes $u$ y $v$ que tenemos que encontrar a partir de las condiciones iniciales

Imponemos por tanto que $a_0=0=u+v$ y que $a_1=1=2u+3v$

In [44]:
u,v = symbols("u,v")
solve([u+v,2*u+3*v-1])

{u: -1, v: 1}

Por lo que $a_n=-2^n+3^n$, como habíamos visto arriba

Veamos un ejemplo de una que no sea homogénea

In [45]:
rsolve(a(n+2)-5*a(n+1)+6*a(n)-n,a(n))

2**n*C0 + 3**n*C1 + C0*n

Y ahora otro ejemplo no lineal

In [46]:
f=a(n)-(n-1)*(a(n-1)+a(n-2))

In [47]:
rsolve(f,a(n),{a(0):1})

factorial(n)

In [48]:
simplify(factorial(n)-(n-1)*(factorial(n-1)+factorial(n-2)))

0

Por último, veamos un ejemplo en el que `rsolve` parece no encontrar una solución satisfactoria

In [1]:
from sympy import *
n=Symbol("n", integer = True)
a=Function('a')
rsolve(a(n)-2*a(n-1)-n,a(n),[1])

2**n/2 + n/2 + 1/2

In [2]:
f = lambda n : (2**n + n + 1)/2

In [3]:
simplify(f(n)-2*f(n-1)-n)

-3*n/2 + 1/2

Para eliminar el término en `n`, vamos restar dos términos consecutivos de la ecuación

In [4]:
a(n)-2*a(n-1)-n-(a(n-1)-2*a(n-2)-(n-1))

a(n) + 2*a(n - 2) - 3*a(n - 1) - 1

In [5]:
rsolve(a(n) + 2*a(n - 2) - 3*a(n - 1) - 1,a(n),[1,3])

3*2**n - n - 2

In [6]:
ff = lambda n: 3*2**n - n - 2

In [7]:
simplify(ff(n)-2*ff(n-1)-n)

0

Por lo tanto, así si que encuentra una solución

Si intentamos "homogeneizar" eliminando el 1, obtenemos

In [8]:
a(n) + 2*a(n - 2) - 3*a(n - 1) - 1 - (a(n-1) + 2*a(n - 3) - 3*a(n - 2) - 1)

a(n) - 2*a(n - 3) + 5*a(n - 2) - 4*a(n - 1)

In [9]:
rsolve(a(n) - 2*a(n - 3) + 5*a(n - 2) - 4*a(n - 1),a(n),[1,3,8])

Que vuelve a dar problemas

Hagámosla a mano

In [10]:
x=Symbol("x")
factor(x**3-4*x**2+5*x-2)

(x - 2)*(x - 1)**2

Por lo que la solución general es de la forma $u2^n+v+n w$

In [11]:
u,v,w = symbols("u,v,w")
s = lambda n:u*2**n+v+n*w
solve([s(0)-1,s(1)-3,s(2)-8],[u,v,w])

{w: -1, v: -2, u: 3}